# Import

In [1]:
import os
import shutil

import random
import numpy as np
import time
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.transforms import CenterCrop, Resize
from PIL import Image

from swinmodel import *

import warnings
warnings.filterwarnings(action='ignore')


/home/chlwnsgur129/anaconda3/envs/ga/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/chlwnsgur129/anaconda3/envs/ga/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")
/home/chlwnsgur129/anaconda3/envs/ga/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


# Hyperparameter Setting

In [2]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':100,
    'LEARNING_RATE':0.001,
    'BATCH_SIZE':4,
    'SEED':42
}

# Fixed RandomSeed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# Data Pre-processing

In [4]:
data_dir = './data'
training_base_dir = os.path.join(data_dir, 'Training')
validation_base_dir = os.path.join(data_dir, 'Validation')

# CustomDataset

In [5]:
class CustomDataset(Dataset):
    def __init__(self, clean_image_paths, noisy_image_paths, transform=None):
        self.clean_image_paths = [os.path.join(clean_image_paths, x) for x in os.listdir(clean_image_paths)]
        self.noisy_image_paths = [os.path.join(noisy_image_paths, x) for x in os.listdir(noisy_image_paths)]
        self.transform = transform
        self.center_crop = CenterCrop(1080)
        self.resize = Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE']))

        # Create a list of (noisy, clean) pairs
        self.noisy_clean_pairs = self._create_noisy_clean_pairs()

    def _create_noisy_clean_pairs(self):
        clean_to_noisy = {}
        for clean_path in self.clean_image_paths:
            clean_id = '_'.join(os.path.basename(clean_path).split('_')[:-1])
            clean_to_noisy[clean_id] = clean_path
        
        noisy_clean_pairs = []
        for noisy_path in self.noisy_image_paths:
            noisy_id = '_'.join(os.path.basename(noisy_path).split('_')[:-1])
            if noisy_id in clean_to_noisy:
                clean_path = clean_to_noisy[noisy_id]
                noisy_clean_pairs.append((noisy_path, clean_path))
            else:
                pass
        
        return noisy_clean_pairs

    def __len__(self):
        return len(self.noisy_clean_pairs)

    def __getitem__(self, index):
        noisy_image_path, clean_image_path = self.noisy_clean_pairs[index]

        noisy_image = Image.open(noisy_image_path).convert("RGB")
        clean_image = Image.open(clean_image_path).convert("RGB")
        
        # Central Crop and Resize
        noisy_image = self.center_crop(noisy_image)
        clean_image = self.center_crop(clean_image)
        noisy_image = self.resize(noisy_image)
        clean_image = self.resize(clean_image)
        
        if self.transform:
            noisy_image = self.transform(noisy_image)
            clean_image = self.transform(clean_image)
        
        return noisy_image, clean_image

# Inference

In [6]:
class CustomDatasetTest(data.Dataset):
    def __init__(self, noisy_image_paths, transform=None):
        self.noisy_image_paths = [os.path.join(noisy_image_paths, x) for x in os.listdir(noisy_image_paths)]
        self.transform = transform

    def __len__(self):
        return len(self.noisy_image_paths)

    def __getitem__(self, index):
        noisy_image_path = self.noisy_image_paths[index]
        noisy_image = load_img(self.noisy_image_paths[index])
        
        # Convert numpy array to PIL image
        if isinstance(noisy_image, np.ndarray):
            noisy_image = Image.fromarray(noisy_image)

        if self.transform:
            noisy_image = self.transform(noisy_image)

        return noisy_image, noisy_image_path


test_transform = Compose([
    ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

def load_img(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [7]:
device = torch.device('cuda:3')
print(device)
# Restormer 모델 인스턴스 생성 및 GPU로 이동
#model = Restormer()#.to(device)
model = SwinIR(img_size=64, 
               patch_size=1, 
               in_chans=3,
               embed_dim=96, 
               depths=[6, 6, 6, 6], 
               num_heads=[6, 6, 6, 6], 
               window_size=7, 
               mlp_ratio=2, 
               upsampler='pixelshuffledirect')

model = nn.DataParallel(model, device_ids=[3]).to(device)
# State dict 불러오기
state_dict = torch.load('b16.pth')

# "module." 제거
from collections import OrderedDict

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] if k.startswith("module.") else k  # "module." 제거
    new_state_dict[name] = v

cuda:3


In [9]:
from PIL import Image
import os
import torch
from collections import OrderedDict
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from model import Restormer
# 모델 로드
model = Restormer()
#model = SwinIR(img_size=64, 
#               patch_size=1, 
#               in_chans=3,
#               embed_dim=96, 
#               depths=[6, 6, 6, 6], 
#               num_heads=[6, 6, 6, 6], 
#               window_size=7, 
#               mlp_ratio=2, 
#               upsampler='pixelshuffledirect')

# GPU 설정
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model.to(device)

# State dict 불러오기 후 "module." 제거
state_dict = torch.load('b16.pth')
new_state_dict = OrderedDict((k[7:] if k.startswith("module.") else k, v) for k, v in state_dict.items())
model.load_state_dict(new_state_dict)

# 테스트 데이터 경로 및 출력 경로 설정
test_data_path = './data/test/Input'
output_path = './data/test/Output'

# 데이터셋 로드 및 전처리
test_dataset = CustomDatasetTest(test_data_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 출력 폴더가 없으면 생성
if not os.path.exists(output_path):
    os.makedirs(output_path)

# 리사이즈 변환 설정
resize_transform = transforms.Resize((224, 224))

# 이미지 denoising 및 저장
for noisy_image, noisy_image_path in test_loader:
    noisy_image = noisy_image.to(device)
    
    with torch.no_grad():
        denoised_image = model(noisy_image)
    
    # denoised_image를 CPU로 이동하여 이미지 저장 준비
    denoised_image = denoised_image.cpu().squeeze(0)
    denoised_image = (denoised_image * 0.5 + 0.5).clamp(0, 1)
    
    # PIL 이미지로 변환 후 리사이즈
    denoised_image = transforms.ToPILImage()(denoised_image)
    denoised_image = resize_transform(denoised_image)  # 224x224로 리사이즈
    
    # 저장 경로 지정 및 저장
    output_filename = noisy_image_path[0] if isinstance(noisy_image_path, (list, tuple)) else noisy_image_path
    denoised_filename = os.path.join(output_path, os.path.basename(output_filename))
    denoised_image.save(denoised_filename)
    
    print(f'Saved denoised image: {denoised_filename}')

RuntimeError: Error(s) in loading state_dict for SwinIR:
	Missing key(s) in state_dict: "conv_first.weight", "conv_first.bias", "patch_embed.norm.weight", "patch_embed.norm.bias", "layers.0.residual_group.blocks.0.norm1.weight", "layers.0.residual_group.blocks.0.norm1.bias", "layers.0.residual_group.blocks.0.attn.relative_position_bias_table", "layers.0.residual_group.blocks.0.attn.relative_position_index", "layers.0.residual_group.blocks.0.attn.qkv.weight", "layers.0.residual_group.blocks.0.attn.qkv.bias", "layers.0.residual_group.blocks.0.attn.proj.weight", "layers.0.residual_group.blocks.0.attn.proj.bias", "layers.0.residual_group.blocks.0.norm2.weight", "layers.0.residual_group.blocks.0.norm2.bias", "layers.0.residual_group.blocks.0.mlp.fc1.weight", "layers.0.residual_group.blocks.0.mlp.fc1.bias", "layers.0.residual_group.blocks.0.mlp.fc2.weight", "layers.0.residual_group.blocks.0.mlp.fc2.bias", "layers.0.residual_group.blocks.1.attn_mask", "layers.0.residual_group.blocks.1.norm1.w

In [ ]:
from torchvision import transforms
test_data_path = './data/test/Input'
output_path = './data/test/Output'

test_dataset = CustomDatasetTest(test_data_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)



if not os.path.exists(output_path):
    os.makedirs(output_path)

for noisy_image, noisy_image_path in test_loader:
    noisy_image = noisy_image.to(device)
    with torch.no_grad():
        denoised_image = model(noisy_image)
    
    # denoised_image를 CPU로 이동하여 이미지 저장
    denoised_image = denoised_image.cpu().squeeze(0)
    denoised_image = (denoised_image * 0.5 + 0.5).clamp(0, 1)
    denoised_image = transforms.ToPILImage()(denoised_image)
    
    resize_transform = transforms.Resize((224, 224))
    # Resize to 224x224
    denoised_image = resize_transform(denoised_image)

    # 저장 경로 지정 및 저장
    output_filename = noisy_image_path[0] if isinstance(noisy_image_path, list) else noisy_image_path
    denoised_filename = output_path + '/' + output_filename[0].split('/')[-1][:-4] + '.jpg'
    denoised_image.save(denoised_filename)
    
    print(f'Saved denoised image: {denoised_filename}')

NameError: name 'CustomDatasetTest' is not defined

In [9]:
from PIL import Image
import os
import torch
from collections import OrderedDict
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

# 모델 로드
model = SwinIR(img_size=64, 
               patch_size=1, 
               in_chans=3,
               embed_dim=96, 
               depths=[6, 6, 6, 6], 
               num_heads=[6, 6, 6, 6], 
               window_size=7, 
               mlp_ratio=2, 
               upsampler='pixelshuffledirect')

# GPU 설정
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model.to(device)

# State dict 불러오기 후 "module." 제거
state_dict = torch.load('run2.pth')
new_state_dict = OrderedDict((k[7:] if k.startswith("module.") else k, v) for k, v in state_dict.items())
model.load_state_dict(new_state_dict)

# 테스트 데이터 경로 및 출력 경로 설정
test_data_path = './data/test/Input'
output_path = './data/test/Output'

# 데이터셋 로드 및 전처리
test_dataset = CustomDatasetTest(test_data_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# 출력 폴더가 없으면 생성
if not os.path.exists(output_path):
    os.makedirs(output_path)

# 리사이즈 변환 설정
resize_transform = transforms.Resize((224, 224))

# 이미지 denoising 및 저장
for noisy_image, noisy_image_path in test_loader:
    noisy_image = noisy_image.to(device)
    
    with torch.no_grad():
        denoised_image = model(noisy_image)
    
    # denoised_image를 CPU로 이동하여 이미지 저장 준비
    denoised_image = denoised_image.cpu().squeeze(0)
    denoised_image = (denoised_image * 0.5 + 0.5).clamp(0, 1)
    
    # PIL 이미지로 변환 후 리사이즈
    denoised_image = transforms.ToPILImage()(denoised_image)
    denoised_image = resize_transform(denoised_image)  # 224x224로 리사이즈
    
    # 저장 경로 지정 및 저장
    output_filename = noisy_image_path[0] if isinstance(noisy_image_path, list) else noisy_image_path
    denoised_filename = os.path.join(output_path, os.path.basename(output_filename).replace('.jpg', '_denoised.jpg'))
    denoised_image.save(denoised_filename)
    
    print(f'Saved denoised image: {denoised_filename}')

TypeError: expected str, bytes or os.PathLike object, not tuple

In [ ]:
model = SwinIR(img_size=64, 
               patch_size=1, 
               in_chans=3,
               embed_dim=96, 
               depths=[6, 6, 6, 6], 
               num_heads=[6, 6, 6, 6], 
               window_size=7, 
               mlp_ratio=2, 
               upsampler='pixelshuffledirect')

# GPU 설정
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
model.to(device)

# State dict 불러오기 후 "module." 제거
state_dict = torch.load('run2.pth')
new_state_dict = OrderedDict((k[7:] if k.startswith("module.") else k, v) for k, v in state_dict.items())
model.load_state_dict(new_state_dict)

test_data_path = './data/test/Input'
output_path = './data/test/Output'

# 데이터셋 로드 및 전처리
test_dataset = CustomDatasetTest(test_data_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

if not os.path.exists(output_path):
    os.makedirs(output_path)
resize_transform = transforms.Resize((224, 224))
# 이미지 denoising 및 저장
for noisy_image, noisy_image_path in test_loader:
    noisy_image = noisy_image.to(device)
    with torch.no_grad():
        denoised_image = model(noisy_image)
    
    # denoised_image를 CPU로 이동하여 이미지 저장
    denoised_image = denoised_image.cpu().squeeze(0)
    denoised_image = (denoised_image * 0.5 + 0.5).clamp(0, 1)
    denoised_image = transforms.ToPILImage()(denoised_image)

    # 저장 경로 지정 및 저장
    output_filename = noisy_image_path[0] if isinstance(noisy_image_path, list) else noisy_image_path
    print(output_filename)
    denoised_filename = output_path + '/' + output_filename[0].split('/')[-1][:-4] + '.jpg'
    denoised_image.save(denoised_filename)
    
    print(f'Saved denoised image: {denoised_filename}')

('./data/test/Input/TEST_000.jpg',)
Saved denoised image: ./data/test/Output/TEST_000.jpg
('./data/test/Input/TEST_001.jpg',)
Saved denoised image: ./data/test/Output/TEST_001.jpg
('./data/test/Input/TEST_002.jpg',)
Saved denoised image: ./data/test/Output/TEST_002.jpg
('./data/test/Input/TEST_003.jpg',)
Saved denoised image: ./data/test/Output/TEST_003.jpg
('./data/test/Input/TEST_004.jpg',)
Saved denoised image: ./data/test/Output/TEST_004.jpg
('./data/test/Input/TEST_005.jpg',)
Saved denoised image: ./data/test/Output/TEST_005.jpg
('./data/test/Input/TEST_006.jpg',)
Saved denoised image: ./data/test/Output/TEST_006.jpg
('./data/test/Input/TEST_007.jpg',)
Saved denoised image: ./data/test/Output/TEST_007.jpg
('./data/test/Input/TEST_008.jpg',)
Saved denoised image: ./data/test/Output/TEST_008.jpg
('./data/test/Input/TEST_009.jpg',)
Saved denoised image: ./data/test/Output/TEST_009.jpg
('./data/test/Input/TEST_010.jpg',)
Saved denoised image: ./data/test/Output/TEST_010.jpg
('./data/t

In [1]:
import PIL

In [9]:
from PIL import Image

# Correct syntax to open the image
image = Image.open('./data/test/Output/TEST_000.jpg')
image.size

(224, 224)

# Submission

In [10]:
def zip_folder(folder_path, output_zip):
    shutil.make_archive(output_zip, 'zip', folder_path)
    print(f"Created {output_zip}.zip successfully.")
zip_folder(output_path, './submission')

Created ./submission.zip successfully.
